In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import sys
import csv
import functools
from functools import reduce
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import re
import string
import statistics

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
#from sklearn import svm
#from sklearn.svm import SVC
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [2]:
udf = pd.read_pickle("./4_COMB_FILE_UNDERSAMPLING")
print(str(udf.shape))
odf = pd.read_pickle("./4_COMB_FILE_OVERSAMPLING")
print(str(odf.shape))
df = pd.read_pickle("./4_COMB_FILE_UNBALANCED.pkl")
print(str(df.shape))

(19732, 49)
(31622, 49)
(25677, 49)


In [3]:
# create bag of words for df
cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(df.CLEAN_FOI_TEXT)
df_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
df_dtm.index = df.index
print(df_dtm.shape)

# train-test split for df
X_df = df_dtm
y_df = df["RECALL_CLASS"]

X_df_train, X_df_test, y_df_train, y_df_test = train_test_split(X_df, y_df, test_size = 0.33, random_state = 0)
print(X_df_train.shape)
print(X_df_test.shape)
print(y_df_train.shape)
print(y_df_test.shape)

(25677, 6205)
(17203, 6205)
(8474, 6205)
(17203,)
(8474,)


In [4]:
# create bag of words for udf
cv_u = CountVectorizer(stop_words='english')
data_cv_u = cv_u.fit_transform(udf.CLEAN_FOI_TEXT)
udf_dtm = pd.DataFrame(data_cv_u.toarray(), columns=cv_u.get_feature_names())
udf_dtm.index = udf.index
print(udf_dtm.shape)

# train-test split for udf
X_u = udf_dtm
y_u = udf["RECALL_CLASS"]

X_u_train, X_u_test, y_u_train, y_u_test = train_test_split(X_u, y_u, test_size = 0.33, random_state = 0)
print(X_u_train.shape)
print(X_u_test.shape)
print(y_u_train.shape)
print(y_u_test.shape)

(19732, 5515)
(13220, 5515)
(6512, 5515)
(13220,)
(6512,)


In [5]:
# create bag of words for odf
cv_o = CountVectorizer(stop_words='english')
data_cv_o = cv_o.fit_transform(odf.CLEAN_FOI_TEXT)
odf_dtm = pd.DataFrame(data_cv_o.toarray(), columns=cv_o.get_feature_names())
odf_dtm.index = odf.index
print(odf_dtm.shape)

# train-test split for odf
X_o = odf_dtm
y_o = odf["RECALL_CLASS"]

X_o_train, X_o_test, y_o_train, y_o_test = train_test_split(X_o, y_o, test_size = 0.33, random_state = 0)
print(X_o_train.shape)
print(X_o_test.shape)
print(y_o_train.shape)
print(y_o_test.shape)

(31622, 6172)
(21186, 6172)
(10436, 6172)
(21186,)
(10436,)


In [6]:
# running naive-bayes classifiers
from sklearn.naive_bayes import GaussianNB

In [7]:
# for unbalanced class data
classifierdf = GaussianNB()
classifierdf.fit(X_df_train, y_df_train)
ydfPredNB = classifierdf.predict(X_df_test)
print("Unbalanced classes")
print(confusion_matrix(y_df_test,ydfPredNB))
print('\n')
print(classification_report(y_df_test,ydfPredNB))

Unbalanced classes
[[3094  193]
 [2993 2194]]


              precision    recall  f1-score   support

           0       0.51      0.94      0.66      3287
           1       0.92      0.42      0.58      5187

    accuracy                           0.62      8474
   macro avg       0.71      0.68      0.62      8474
weighted avg       0.76      0.62      0.61      8474



In [8]:
# for undersampled class data
classifieru = GaussianNB()
classifieru.fit(X_u_train, y_u_train)
yuPredNB = classifieru.predict(X_u_test)
print("Undersampled classes")
print(confusion_matrix(y_u_test,yuPredNB))
print('\n')
print(classification_report(y_u_test,yuPredNB))

Undersampled classes
[[3073  164]
 [1801 1474]]


              precision    recall  f1-score   support

           0       0.63      0.95      0.76      3237
           1       0.90      0.45      0.60      3275

    accuracy                           0.70      6512
   macro avg       0.77      0.70      0.68      6512
weighted avg       0.77      0.70      0.68      6512



In [9]:
# for oversampled class data
classifier = GaussianNB()
classifier.fit(X_o_train, y_o_train)
yoPredNB = classifier.predict(X_o_test)
print("Oversampled (x) classes")
print(confusion_matrix(y_o_test,yoPredNB))
print('\n')
print(classification_report(y_o_test,yoPredNB))

Oversampled (x) classes
[[5142  152]
 [3089 2053]]


              precision    recall  f1-score   support

           0       0.62      0.97      0.76      5294
           1       0.93      0.40      0.56      5142

    accuracy                           0.69     10436
   macro avg       0.78      0.69      0.66     10436
weighted avg       0.78      0.69      0.66     10436



In [10]:
# running descision tree classifiers
from sklearn.tree import DecisionTreeClassifier

In [11]:
# for unbalanced class data
classifierdf = DecisionTreeClassifier()
classifierdf.fit(X_df_train, y_df_train)
ydfPredDT = classifierdf.predict(X_df_test)
print("Unbalanced classes")
print(confusion_matrix(y_df_test,ydfPredDT))
print('\n')
print(classification_report(y_df_test,ydfPredDT))

Unbalanced classes
[[3083  204]
 [ 241 4946]]


              precision    recall  f1-score   support

           0       0.93      0.94      0.93      3287
           1       0.96      0.95      0.96      5187

    accuracy                           0.95      8474
   macro avg       0.94      0.95      0.94      8474
weighted avg       0.95      0.95      0.95      8474



In [12]:
# for undersampled class data
classifieru = DecisionTreeClassifier()
classifieru.fit(X_u_train, y_u_train)
yuPredDT = classifieru.predict(X_u_test)
print("Undersampled classes")
print(confusion_matrix(y_u_test,yuPredDT))
print('\n')
print(classification_report(y_u_test,yuPredDT))

Undersampled classes
[[3035  202]
 [ 215 3060]]


              precision    recall  f1-score   support

           0       0.93      0.94      0.94      3237
           1       0.94      0.93      0.94      3275

    accuracy                           0.94      6512
   macro avg       0.94      0.94      0.94      6512
weighted avg       0.94      0.94      0.94      6512



In [13]:
# for oversampled class data
classifier = DecisionTreeClassifier()
classifier.fit(X_o_train, y_o_train)
yoPredDT = classifier.predict(X_o_test)
print("Oversampled (x) classes")
print(confusion_matrix(y_o_test,yoPredDT))
print('\n')
print(classification_report(y_o_test,yoPredDT))

Oversampled (x) classes
[[5094  200]
 [ 277 4865]]


              precision    recall  f1-score   support

           0       0.95      0.96      0.96      5294
           1       0.96      0.95      0.95      5142

    accuracy                           0.95     10436
   macro avg       0.95      0.95      0.95     10436
weighted avg       0.95      0.95      0.95     10436



In [14]:
# running KNN classifiers
from sklearn.neighbors import KNeighborsClassifier

In [15]:
# for unbalanced class data
classifierdf = KNeighborsClassifier(n_neighbors=1)
classifierdf.fit(X_df_train, y_df_train)
ydfPredKNN = classifierdf.predict(X_df_test)
print("Unbalanced classes")
print(confusion_matrix(y_df_test,ydfPredKNN))
print('\n')
print(classification_report(y_df_test,ydfPredKNN))

Unbalanced classes
[[3143  144]
 [ 351 4836]]


              precision    recall  f1-score   support

           0       0.90      0.96      0.93      3287
           1       0.97      0.93      0.95      5187

    accuracy                           0.94      8474
   macro avg       0.94      0.94      0.94      8474
weighted avg       0.94      0.94      0.94      8474



In [16]:
# for undersampled class data
classifieru = KNeighborsClassifier(n_neighbors=1)
classifieru.fit(X_u_train, y_u_train)
yuPredKNN = classifieru.predict(X_u_test)
print("Undersampled classes")
print(confusion_matrix(y_u_test,yuPredKNN))
print('\n')
print(classification_report(y_u_test,yuPredKNN))

Undersampled classes
[[3132  105]
 [ 261 3014]]


              precision    recall  f1-score   support

           0       0.92      0.97      0.94      3237
           1       0.97      0.92      0.94      3275

    accuracy                           0.94      6512
   macro avg       0.94      0.94      0.94      6512
weighted avg       0.94      0.94      0.94      6512



In [17]:
# for oversampled class data
classifier = KNeighborsClassifier(n_neighbors=1)
classifier.fit(X_o_train, y_o_train)
yoPredKNN = classifier.predict(X_o_test)
print("Oversampled (x) classes")
print(confusion_matrix(y_o_test,yoPredKNN))
print('\n')
print(classification_report(y_o_test,yoPredKNN))

Oversampled (x) classes
[[5160  134]
 [ 388 4754]]


              precision    recall  f1-score   support

           0       0.93      0.97      0.95      5294
           1       0.97      0.92      0.95      5142

    accuracy                           0.95     10436
   macro avg       0.95      0.95      0.95     10436
weighted avg       0.95      0.95      0.95     10436



In [18]:
# running random forest classifier
from sklearn.ensemble import RandomForestClassifier

In [19]:
# for unbalanced class data
classifierdf = RandomForestClassifier(n_estimators=100)
classifierdf.fit(X_df_train, y_df_train)
ydfPredRF = classifierdf.predict(X_df_test)
print("Unbalanced classes")
print(confusion_matrix(y_df_test,ydfPredRF))
print('\n')
print(classification_report(y_df_test,ydfPredRF))

Unbalanced classes
[[3114  173]
 [ 152 5035]]


              precision    recall  f1-score   support

           0       0.95      0.95      0.95      3287
           1       0.97      0.97      0.97      5187

    accuracy                           0.96      8474
   macro avg       0.96      0.96      0.96      8474
weighted avg       0.96      0.96      0.96      8474



In [20]:
# for undersampled class data
classifieru = RandomForestClassifier(n_estimators=100)
classifieru.fit(X_u_train, y_u_train)
yuPredRF = classifieru.predict(X_u_test)
print("Undersampled classes")
print(confusion_matrix(y_u_test,yuPredRF))
print('\n')
print(classification_report(y_u_test,yuPredRF))

Undersampled classes
[[3083  154]
 [ 137 3138]]


              precision    recall  f1-score   support

           0       0.96      0.95      0.95      3237
           1       0.95      0.96      0.96      3275

    accuracy                           0.96      6512
   macro avg       0.96      0.96      0.96      6512
weighted avg       0.96      0.96      0.96      6512



In [21]:
# for oversampled class data
classifier = RandomForestClassifier(n_estimators=100)
classifier.fit(X_o_train, y_o_train)
yoPredRF = classifier.predict(X_o_test)
print("Oversampled (x) classes")
print(confusion_matrix(y_o_test,yoPredRF))
print('\n')
print(classification_report(y_o_test,yoPredRF))

Oversampled (x) classes
[[5124  170]
 [ 193 4949]]


              precision    recall  f1-score   support

           0       0.96      0.97      0.97      5294
           1       0.97      0.96      0.96      5142

    accuracy                           0.97     10436
   macro avg       0.97      0.97      0.97     10436
weighted avg       0.97      0.97      0.97     10436



In [22]:
# running stochastic gradient gescent classifier
from sklearn.linear_model import SGDClassifier

In [23]:
# for unbalanced class data
classifierdf = SGDClassifier()
classifierdf.fit(X_df_train, y_df_train)
ydfPredSGDC = classifierdf.predict(X_df_test)
print("Unbalanced classes")
print(confusion_matrix(y_df_test,ydfPredSGDC))
print('\n')
print(classification_report(y_df_test,ydfPredSGDC))

Unbalanced classes
[[3166  121]
 [ 231 4956]]


              precision    recall  f1-score   support

           0       0.93      0.96      0.95      3287
           1       0.98      0.96      0.97      5187

    accuracy                           0.96      8474
   macro avg       0.95      0.96      0.96      8474
weighted avg       0.96      0.96      0.96      8474



In [24]:
# for undersampled class data
classifieru = SGDClassifier()
classifieru.fit(X_u_train, y_u_train)
yuPredSGDC = classifieru.predict(X_u_test)
print("Undersampled classes")
print(confusion_matrix(y_u_test,yuPredSGDC))
print('\n')
print(classification_report(y_u_test,yuPredSGDC))

Undersampled classes
[[3078  159]
 [ 174 3101]]


              precision    recall  f1-score   support

           0       0.95      0.95      0.95      3237
           1       0.95      0.95      0.95      3275

    accuracy                           0.95      6512
   macro avg       0.95      0.95      0.95      6512
weighted avg       0.95      0.95      0.95      6512



In [25]:
# for oversampled class data
classifier = SGDClassifier()
classifier.fit(X_o_train, y_o_train)
yoPredSGDC = classifier.predict(X_o_test)
print("Oversampled (x) classes")
print(confusion_matrix(y_o_test,yoPredSGDC))
print('\n')
print(classification_report(y_o_test,yoPredSGDC))

Oversampled (x) classes
[[5115  179]
 [ 250 4892]]


              precision    recall  f1-score   support

           0       0.95      0.97      0.96      5294
           1       0.96      0.95      0.96      5142

    accuracy                           0.96     10436
   macro avg       0.96      0.96      0.96     10436
weighted avg       0.96      0.96      0.96     10436



In [26]:
# running gradient boosting classifier
from sklearn.ensemble import GradientBoostingClassifier

In [27]:
# for unbalanced class data
classifierdf = GradientBoostingClassifier()
classifierdf.fit(X_df_train, y_df_train)
ydfPredGBC = classifierdf.predict(X_df_test)
print("Unbalanced classes")
print(confusion_matrix(y_df_test,ydfPredGBC))
print('\n')
print(classification_report(y_df_test,ydfPredGBC))

Unbalanced classes
[[3077  210]
 [ 138 5049]]


              precision    recall  f1-score   support

           0       0.96      0.94      0.95      3287
           1       0.96      0.97      0.97      5187

    accuracy                           0.96      8474
   macro avg       0.96      0.95      0.96      8474
weighted avg       0.96      0.96      0.96      8474



In [28]:
# for undersampled class data
classifieru = GradientBoostingClassifier()
classifieru.fit(X_u_train, y_u_train)
yuPredGBC = classifieru.predict(X_u_test)
print("Undersampled classes")
print(confusion_matrix(y_u_test,yuPredGBC))
print('\n')
print(classification_report(y_u_test,yuPredGBC))

Undersampled classes
[[3050  187]
 [ 137 3138]]


              precision    recall  f1-score   support

           0       0.96      0.94      0.95      3237
           1       0.94      0.96      0.95      3275

    accuracy                           0.95      6512
   macro avg       0.95      0.95      0.95      6512
weighted avg       0.95      0.95      0.95      6512



In [29]:
# for oversampled class data
classifier = GradientBoostingClassifier()
classifier.fit(X_o_train, y_o_train)
yoPredGBC = classifier.predict(X_o_test)
print("Oversampled (x) classes")
print(confusion_matrix(y_o_test,yoPredGBC))
print('\n')
print(classification_report(y_o_test,yoPredGBC))

Oversampled (x) classes
[[5000  294]
 [ 202 4940]]


              precision    recall  f1-score   support

           0       0.96      0.94      0.95      5294
           1       0.94      0.96      0.95      5142

    accuracy                           0.95     10436
   macro avg       0.95      0.95      0.95     10436
weighted avg       0.95      0.95      0.95     10436

